<a href="https://colab.research.google.com/github/raguram/eva/blob/master/S15/DenseDepth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Google Drive

In [2]:
from os.path import join
from google.colab import drive

ROOT = "/content/drive"
drive.mount(ROOT, force_remount=True)

Mounted at /content/drive


In [3]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-077b01c1-c876-80bb-c740-63d963a28b69)


## Downgrade the version of tensor flow

In [4]:
%tensorflow_version 1.13

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.13`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


## Clone the DenseDepth repo

In [4]:
!git clone https://github.com/ialhashim/DenseDepth.git

Cloning into 'DenseDepth'...
remote: Enumerating objects: 235, done.
remote: Total 235 (delta 0), reused 0 (delta 0), pack-reused 235
Receiving objects: 100% (235/235), 11.80 MiB | 6.64 MiB/s, done.
Resolving deltas: 100% (115/115), done.


## Download weights

In [5]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./DenseDepth/nyu.h5

--2020-05-10 12:42:16--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.89.139
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.89.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘./DenseDepth/nyu.h5’

./DenseDepth/nyu.h5 100%[===================>] 164.89M  11.9MB/s    in 16s     

2020-05-10 12:42:33 (10.5 MB/s) - ‘./DenseDepth/nyu.h5’ saved [172897376/172897376]



## Download and unzip the data set 

In [6]:
!unzip /content/drive/My\ Drive/Colab\ Notebooks/workspace/data/Assignment15/data.zip -d /content/input

Streaming output truncated to the last 5000 lines.
 extracting: /content/input/fg_bg/Image_477101.jpg  
 extracting: /content/input/fg_bg/Image_477102.jpg  
 extracting: /content/input/fg_bg/Image_477103.jpg  
 extracting: /content/input/fg_bg/Image_477104.jpg  
 extracting: /content/input/fg_bg/Image_477105.jpg  
 extracting: /content/input/fg_bg/Image_477106.jpg  
 extracting: /content/input/fg_bg/Image_477107.jpg  
 extracting: /content/input/fg_bg/Image_477108.jpg  
 extracting: /content/input/fg_bg/Image_477109.jpg  
 extracting: /content/input/fg_bg/Image_477110.jpg  
 extracting: /content/input/fg_bg/Image_477111.jpg  
 extracting: /content/input/fg_bg/Image_477112.jpg  
 extracting: /content/input/fg_bg/Image_477113.jpg  
 extracting: /content/input/fg_bg/Image_477114.jpg  
 extracting: /content/input/fg_bg/Image_477115.jpg  
 extracting: /content/input/fg_bg/Image_477116.jpg  
 extracting: /content/input/fg_bg/Image_477117.jpg  
 extracting: /content/input/fg_bg/Image_477118.j

## Import 

In [5]:
import sys
sys.path.append("/content/DenseDepth")

import os
import glob
import argparse
import matplotlib

# Keras / TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from keras.models import load_model
from layers import BilinearUpSampling2D
from utils import predict, load_images, display_images
from matplotlib import pyplot as plt
from PIL import Image

Using TensorFlow backend.


## Setup Arguments




In [0]:
model_path = "/content/DenseDepth/nyu.h5"

## Load model 

In [7]:
# Custom object needed for inference and training
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}

print('Loading model...')

# Load model into GPU / CPU
model = load_model(model_path, custom_objects=custom_objects, compile=False)

print('\nModel loaded ({0}).'.format(model_path))

Loading model...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.




Model loaded (/content/DenseDepth/nyu.h5).


## Utility Methods

In [0]:
import numpy as np 
import imageio
from io import BytesIO

def showImages(images, cols=10, figSize=(15, 15)):
    """
    Shows images with its labels. Expected PIL Image. 
    """
    figure = plt.figure(figsize=figSize)
    num_of_images = len(images)
    rows = np.ceil(num_of_images / float(cols))
    for index in range(0, num_of_images):
        plt.subplot(rows, cols, index + 1)
        plt.axis('off')
        plt.imshow(images[index].squeeze(), cmap='gray')

def persistImagesToZip(images, fileNames, folder, zipFile, extn="jpg"): 

    plasma = plt.get_cmap('gray')

    names=[]
    for i, img in enumerate(images):   
      rescaled = img[:,:,0]
      rescaled = rescaled - np.min(rescaled)
      rescaled = rescaled / np.max(rescaled)
      gray_img = plasma(rescaled)[:,:,:3]
      depth_map = Image.fromarray(np.uint8(gray_img*255)).convert("L").resize((224,224))
      with BytesIO() as buf:
        depth_map.save(buf, format="JPEG")
        name = f"{folder}/{fileNames[i]}"
        zipFile.writestr(name, buf.getvalue())
        names.append(name)
    return names

## Utility to iteratively predict and add to the zip

In [0]:
from os import listdir
import math
from skimage.transform import resize
from datetime import datetime
from zipfile import ZipFile

def predictBatch(input, inputFolder, outputZipFolder, zipFile): 

  inputFiles = [join(inputFolder, f) for f in input]
  inputs = load_images(inputFiles)
  outputs = predict(model, inputs)
  outputs = outputs * 255
  outputs = outputs.astype(np.uint8)
  names = persistImagesToZip(outputs, input, outputZipFolder, zipFile)
  return names
  
def iterative_predict(folder, output_folder, batchSize=1000, startIdx=0, numberOfBatches=10000): 
  
  files = listdir(folder)
  print(f"Total number of files: {len(files)}")

  files = files[startIdx:]
  print(f"Start idx is {startIdx}. Number of files to be predicted: {len(files)}")

  for batchNumber in range(min(math.ceil(len(files)/batchSize), numberOfBatches)): 
    startTime = datetime.now()
    zipFilename = f"{output_folder}/fg_bg_depth_{batchNumber}.zip"
    outputZip = ZipFile(zipFilename, mode="a")
    batchStartIdx = batchNumber * batchSize
    names = predictBatch(files[batchStartIdx: batchStartIdx + batchSize], folder, "fg_bg_depth", outputZip) 
    outputZip.close()
    print(f"Processed batch {batchNumber} into {zipFilename}. Time Taken: {datetime.now() - startTime}")

## Predict Depth 

In [10]:
iterative_predict("/content/input/fg_bg", "/content/drive/My Drive/Colab Notebooks/workspace/data/Assignment15/DepthMask/output", 1000)


Total number of files: 479600
Start idx is 0. Number of files to be predicted: 479600

Processed batch 0 into /content/drive/My Drive/Colab Notebooks/workspace/data/Assignment15/DepthMask/output/fg_bg_depth_0.zip. Time Taken: 0:00:44.273475
Processed batch 1 into /content/drive/My Drive/Colab Notebooks/workspace/data/Assignment15/DepthMask/output/fg_bg_depth_1.zip. Time Taken: 0:00:29.350047
Processed batch 2 into /content/drive/My Drive/Colab Notebooks/workspace/data/Assignment15/DepthMask/output/fg_bg_depth_2.zip. Time Taken: 0:00:26.915397
Processed batch 3 into /content/drive/My Drive/Colab Notebooks/workspace/data/Assignment15/DepthMask/output/fg_bg_depth_3.zip. Time Taken: 0:00:25.590942
Processed batch 4 into /content/drive/My Drive/Colab Notebooks/workspace/data/Assignment15/DepthMask/output/fg_bg_depth_4.zip. Time Taken: 0:00:25.359893
Processed batch 5 into /content/drive/My Drive/Colab Notebooks/workspace/data/Assignment15/DepthMask/output/fg_bg_depth_5.zip. Time Taken: 0:00

In [14]:
# Collate the zip files 

zips = [join("/content/drive/My Drive/Colab Notebooks/workspace/data/Assignment15/DepthMask/output", f) for f in listdir("/content/drive/My Drive/Colab Notebooks/workspace/data/Assignment15/DepthMask/output")]
with ZipFile("/content/drive/My Drive/Colab Notebooks/workspace/data/Assignment15/DepthMask/fg_bg_depth.zip", 'a') as oZip:
    for fname in zips[1:]:
        print(f"Adding {fname} to the zip.")
        with ZipFile(fname, 'r') as zf:
          for n in zf.namelist():
            oZip.writestr(n, zf.open(n).read())

Adding /content/drive/My Drive/Colab Notebooks/workspace/data/Assignment15/DepthMask/output/fg_bg_depth_1.zip to the zip.
Adding /content/drive/My Drive/Colab Notebooks/workspace/data/Assignment15/DepthMask/output/fg_bg_depth_2.zip to the zip.
Adding /content/drive/My Drive/Colab Notebooks/workspace/data/Assignment15/DepthMask/output/fg_bg_depth_3.zip to the zip.
Adding /content/drive/My Drive/Colab Notebooks/workspace/data/Assignment15/DepthMask/output/fg_bg_depth_4.zip to the zip.
Adding /content/drive/My Drive/Colab Notebooks/workspace/data/Assignment15/DepthMask/output/fg_bg_depth_5.zip to the zip.
Adding /content/drive/My Drive/Colab Notebooks/workspace/data/Assignment15/DepthMask/output/fg_bg_depth_6.zip to the zip.
Adding /content/drive/My Drive/Colab Notebooks/workspace/data/Assignment15/DepthMask/output/fg_bg_depth_7.zip to the zip.
Adding /content/drive/My Drive/Colab Notebooks/workspace/data/Assignment15/DepthMask/output/fg_bg_depth_8.zip to the zip.
Adding /content/drive/My